
this notebook contains all plot used in our report.

In [2]:
import numpy as np
import torch
import scipy
import os
import matplotlib.pyplot as plt

## 1. MNIST experiments

## Imports

In [3]:
from ManifoldProjectLoss import ManifoldProjection
import torchvision
import torchvision.transforms as transforms
from sklearn.neighbors import NearestNeighbors



## Load MNIST data

In [4]:


# Download MNIST training
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True)

#  Convert training data to NumPy arrays
train_images = trainset.data.numpy()   # Shape: (60000, 28, 28)
train_labels = trainset.targets.numpy()  # Shape: (60000,)
test_images = testset.data.numpy()    # Shape: (10000, 28, 28)
test_labels = testset.targets.numpy()  # Shape: (10000,)

# flatten images into normalized vectors
X_train = train_images.reshape(train_images.shape[0], -1)  / 255 # Shape: (60000, 784)
X_test = test_images.reshape(test_images.shape[0], -1)  /255   # Shape: (10000, 784)
Y_train = train_labels
Y_test = test_labels
# shapes print
print(f"Train images shape: {X_train.shape}, Train labels shape: {Y_train.shape}")
print(f"Test images shape: {X_test.shape}, Test labels shape: {Y_test.shape}")

def get_image(flatten_img):
    return flatten_img.reshape(28, 28)

index_4_test = np.where(test_labels==4)[0]
index_8_test = np.where(test_labels==8)[0]
index_9_test = np.where(test_labels==9)[0]


Train images shape: (60000, 784), Train labels shape: (60000,)
Test images shape: (10000, 784), Test labels shape: (10000,)


In [5]:
    
def gradient_descent_on_manifold_cost_function(start, end, data):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    lr=1e-2
    img_x = start
    img_y = end
    x = torch.tensor(img_x).to(device).requires_grad_(True)# 7
    y = torch.tensor(img_y).to(device) # 0
    
    manifold_proj = ManifoldProjection(torch.tensor(data), k_neighbor=15, closing_manifold_ratio=0.1, manifold_ratio=0.8, device=device)
    optimizer = torch.optim.SGD([x], lr=lr)
    num_iterations = 104
    history = []
    # Gradient descent loop
    for i in range(num_iterations):
        optimizer.zero_grad()
        
        loss = manifold_proj(x,y)
        loss.backward()
        optimizer.step()
        
        history.append(x.clone().detach().to("cpu").numpy())
    history = np.array(history)
    return history

In [6]:
def save_plots(history, name, labels = None):
    os.makedirs('./plots',exist_ok=True )
    os.makedirs(f'./plots/{name}', exist_ok=True )
    if labels is None:
        labels = range(len(history))
    iterator = zip(history, labels)
    for flatten_img,label in iterator:
        img = flatten_img.reshape(28,28)
        plt.imsave(f"./plots/{name}/mnist_{name}_{label}.png", img)
def check_plots(history):
    fig, axes = plt.subplots(len(history) // 4, 2, figsize=(50, 400))
    axes = axes.flatten()
    for k,ax in enumerate(axes):
        img = history[2*k].reshape(28,28)
        ax.imshow(img, cmap='gray')
        ax.set_title(f"{2*k}th iteration")

### Experiment transformation 7 to 0

In [7]:
history = gradient_descent_on_manifold_cost_function(X_test[0], X_test[3], X_train)
save_plots(history[::10], "7_to_0", labels= [10*k for k in range(len(history)//10)])

## Experiment transformation 8 to 9

In [8]:
img_x = X_test[index_8_test[1]]
img_y = X_test[index_9_test[0]]
history = gradient_descent_on_manifold_cost_function(img_x, img_y, X_train)
save_plots(history[::10], "8_to_9", labels= [10*k for k in range(len(history)//10)])